<img src="../imgs/logo.png" width="20%" align="right" style="margin:0px 20px">


# Evolutionary Computation

## 5.3 Deep Neuroevolution

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" align="left" src="https://i.creativecommons.org/l/by-sa/4.0/80x15.png" /></a>&nbsp;| Dennis G. Wilson | <a href="https://d9w.github.io/evolution/">https://d9w.github.io/evolution/</a>

In [3]:
using PyCall
using Conda

In [4]:
Conda.add_channel("conda-forge")

┌ Info: Running `conda config --add channels conda-forge --file 'C:\Users\benny\.julia\conda\3\condarc-julia.yml' --force` in root environment
└ @ Conda C:\Users\benny\.julia\packages\Conda\3rPhK\src\Conda.jl:113


In [5]:
Conda.add("gym")

┌ Info: Running `conda install -y gym` in root environment
└ @ Conda C:\Users\benny\.julia\packages\Conda\3rPhK\src\Conda.jl:113


Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\benny\.julia\conda\3

  added / updated specs:
    - gym


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.1 |       hecc5488_0         184 KB  conda-forge
    certifi-2020.4.5.1         |   py37hc8dfbb8_0         150 KB  conda-forge
    cloudpickle-1.2.2          |             py_1          23 KB  conda-forge
    conda-4.8.3                |   py37hc8dfbb8_1         3.1 MB  conda-forge
    ffmpeg-4.2                 |       h6538335_0        23.4 MB  conda-forge
    freetype-2.10.1            |       ha9979f8_0         481 KB  conda-forge
    future-0.18.2              |   py37hc8dfbb8_1         732 KB  conda-forge
    gym-0.17.1                 |   py37hc8dfbb8_0         1.8 MB  conda-forge
    openssl-1.1.1g             |       he774522_0         5.7 MB  conda-forge
 

In [6]:
import Random
Random.seed!(1234);

In [7]:
include("cmaes.jl");

In [8]:
struct FCLayer
    w::Array{Float64}
    b::Array{Float64}
end

struct SimpleANN
    l1::FCLayer
    l2::FCLayer
    out::FCLayer
end

In [9]:
function SimpleANN(input::Int, N1::Int, N2::Int, output::Int)
    l1 = FCLayer(zeros(N1, input), zeros(N1))
    l2 = FCLayer(zeros(N2, N1), zeros(N2))
    out = FCLayer(zeros(output, N2), zeros(output))
    SimpleANN(l1, l2, out)
end

SimpleANN

In [27]:
ann = SimpleANN(5, 64, 64, 4);

In [29]:
function compute(inputs::Array{Float64}, ann::SimpleANN)
    x = ann.l1.w * inputs .+ ann.l1.b
    x = ann.l2.w * x .+ ann.l2.b
    x = ann.out.w * x .+ ann.out.b
    x
end

compute (generic function with 1 method)

In [30]:
compute(zeros(5), ann)

4-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0

In [31]:
gym = pyimport("gym")

PyObject <module 'gym' from 'C:\\Users\\benny\\.julia\\conda\\3\\lib\\site-packages\\gym\\__init__.py'>

In [32]:
env = gym.make("CartPole-v1")
n_in = 4
n_out = 2;

In [33]:
function play_env(ann::SimpleANN; render=false)
    env = gym.make("CartPole-v1")
    env.seed(0)
    obs = env.reset()
    total_reward = 0.0
    done = false
    
    while ~done
        action = argmax(compute(obs, ann))-1
        obs, reward, done, _ = env.step(action)
        if render
            env.render()
        end
        total_reward += reward
    end
    env.close()
    env = nothing
    Base.GC.gc()
    total_reward
end

play_env (generic function with 1 method)

In [34]:
ann = SimpleANN(n_in, 5, 5, n_out)
play_env(ann; render=true)

9.0

In [35]:
play_env(ann)

9.0

In [36]:
function genes_to_ann(genes::Array{Float64})
    ann = SimpleANN(n_in, 5, 5, n_out)
    layers = [ann.l1.w, ann.l1.b, ann.l2.w, ann.l2.b, ann.out.w, ann.out.b]
    L = 1
    j = 1
    for i in eachindex(genes)
        if j > length(layers[L])
            L += 1
            j = 1
        end
        layers[L][j] = genes[i]
        j += 1
    end
    ann
end

genes_to_ann (generic function with 1 method)

In [19]:
function objective(genes::Array{Float64})
    ann = genes_to_ann(genes)
    -play_env(ann)
end

objective (generic function with 1 method)

In [39]:
N = n_in*5 + 5 + 5*5 + 5 + 5*n_out + n_out

67

In [51]:
ann = genes_to_ann(randn(N))

SimpleANN(FCLayer([-0.9307903502241032 -0.4594668820739534 -0.23076204525894994 -1.3121181439677128; 0.02398694503978947 2.495530497059298 -0.6789845893666425 -0.7371391184055938; … ; -0.13784993345578042 0.5484086472976889 1.049314029870312 -1.0097052645063562; 0.25159802993514807 -1.340938496352466 0.48921526283301986 -1.5954526399835631], [-1.3597935780188675, 0.9657203642622076, 0.34771017484272176, 0.7531851135533174, 0.1207027440992206]), FCLayer([-0.10959191737371257 -1.1564670796086132 … 0.6131021338178818 0.2892796477704503; -0.01643502570303541 -0.3825304024317929 … 0.9969019758095559 0.7203584369807631; … ; 1.4252491005265755 0.1698628181588662 … -0.6873360226777636 -1.1278388887125537; -0.4142929815732144 -0.15111273705384007 … -1.6586281050026699 0.12217824211565345], [0.009365517396756812, 2.1573875259279527, -0.6154892322147331, -0.4152571856693953, -1.8584758431623767]), FCLayer([-0.6390608824201006 -0.4428010710274583 … 0.6128337260143029 0.00878216842259964; -0.090038

In [52]:
play_env(ann)

10.0

In [49]:
play_env(ann)

10.0

In [53]:
c = CMAES(N=N, µ=10, λ=30, τ=sqrt(N), τ_c=N^2, τ_σ=sqrt(N))
for i in 1:5
    step!(c, objective)
    println(i, " ", maximum(.-c.F_λ))
end

1 171.0
2 44.0
3 46.0
4 194.0
5 500.0


In [54]:
best = nothing
best_fit = -Inf
c = CMAES(N=N, µ=10, λ=30, τ=sqrt(N), τ_c=N^2, τ_σ=sqrt(N))
for i in 1:20
    step!(c, objective)
    bestind = argmin(c.F_λ)
    maxfit = -c.F_λ[bestind]
    println(i, " ", maxfit)
    if maxfit > best_fit
        best = copy(c.offspring[bestind])
        best_fit = maxfit
    end
    if best_fit == 500
        break
    end
end

1 51.0
2 113.0
3 457.0
4 71.0
5 136.0
6 500.0


In [55]:
ann = genes_to_ann(best)
play_env(ann; render=true)

500.0